In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import numpy as np
from contextily import add_basemap
from libpysal import weights
import time
import shapely
import heapq

In [2]:
roads = gpd.read_file('road_vertices_3857_subset.zip')[['LengthM', 'geometry', 'FULLNAME', 'MTFCC', 'StartX', 'StartY']]
roads = roads[(roads['MTFCC'] == 'S1100') | (roads['MTFCC'] == 'S1200')]

In [18]:
def bellmanford(Graph, source = 963):
    # 963 = index of Gainesville Fire Station
    list(Graph.nodes(data=True))[source][1]['dist'] = 0
    for i in range(len(list(G.nodes))):
      for from_vertex in Graph.nodes:
          neighbors = Graph.__getitem__(from_vertex)
          for to_vertex in neighbors:
              du = Graph.nodes[from_vertex]['dist']
              wt = neighbors[to_vertex][0]['LengthM']
              dv = Graph.nodes[to_vertex]['dist']
              if du + wt < dv:
                  Graph.nodes[to_vertex]['dist'] = du + wt
                  Graph.nodes[to_vertex]['pred'] = from_vertex
    return Graph
    

# based off of pseudocode from discussion slides
# repeat V - 1 times
# for each edge(from u, to v) with weight w in edges do:
#     if dist[u] + w < dist[v] then
#         dist[v] = dist[u] + w
#         pred[v] = u

In [10]:
def dijkstra(Graph, start=963):
    # Initialize everything
    # Start is the index of Gainesville's Fire Station
    list(Graph.nodes(data=True))[start][1]['dist'] = 0
    # Initialize priority queue
    pq = [(0, list(Graph.nodes)[start])]
    visitedNodes = set()

    while pq:
        current_distance, current = heapq.heappop(pq)
        if current in visitedNodes:
            continue
        visitedNodes.add(current)
        currentNeighbors = Graph.__getitem__(current)
        for neighboringNodes in currentNeighbors:
            weight = currentNeighbors[neighboringNodes][0]['LengthM']
            distance = current_distance + weight

            # If distance is less than the current distance, update
            if distance < Graph.nodes[neighboringNodes]['dist']:
                Graph.nodes[neighboringNodes]['dist'] = distance
                Graph.nodes[neighboringNodes]['pred'] = current
                heapq.heappush(pq, (distance, neighboringNodes))
    return Graph

In [13]:
# Example: lon = -82.3111426 lat = 29.6520652
print("Welcome to BlazeNav! We will help you determine the closest path from the Gainesville Fire Station to a fire you've just located.")
print("-------------------------------------------------------------")
print("Please enter the coordinates of your fire in decimal degrees.")
fire_lon = input("Enter longtiude of fire: ")
fire_lat = input("Enter latitude of fire: ")
print("-------------------------------------------------------------")
print("Please select which algorithm you wish to run (enter 1 or 2).")
print("1. Bellman-Ford algorithm")
print("2. Dijkstra's algorithm")
option = input()

# Find the closest street to the input coordinates and use that as the destination
try:
  # References https://stackoverflow.com/questions/70626218/how-to-find-the-nearest-linestring-to-a-point
  fire_point = gpd.GeoDataFrame(geometry=[shapely.wkt.loads(f"POINT ({fire_lon} {fire_lat})")]).set_crs('EPSG:4326').to_crs('EPSG:3857')
  distance = gpd.sjoin_nearest(fire_point, roads).merge(roads, left_on='index_right', right_index=True)
  distance['distance'] = distance.apply(lambda row: row['geometry_x'].distance(row['geometry_y']), axis=1)
  closest_node_X = distance.iloc[0]['StartX_y']
  closest_node_Y = distance.iloc[0]['StartY_y']
  print("We have identified the location. BlazeNav is now searching for the fastest route from the Gainesville Fire Station #1 to the road nearest to the fire you inputted.")
except:
  print("Coordinates not found in or near Alachua County. Please enter new coordinates.")
  fire_lat = input()
  fire_lon = input()

Welcome to BlazeNav! We will help you determine the closest path from the Gainesville Fire Station to a fire you've just located.
-------------------------------------------------------------
Please enter the coordinates of your fire in decimal degrees.


Enter longtiude of fire:  -82.3111426
Enter latitude of fire:  29.6520652


-------------------------------------------------------------
Please select which algorithm you wish to run (enter 1 or 2).
1. Bellman-Ford algorithm
2. Dijkstra's algorithm


 1


We have identified the location. BlazeNav is now searching for the fastest route from the Gainesville Fire Station #1 to the road nearest to the fire you inputted.


In [14]:
fire_lonlat = (fire_lon, fire_lat)

In [15]:
if option == 1:
    result = bellmanford(G)
    # findpath(fire_lonlat, result)

In [17]:
if option == 2:
    result = dijkstra(G)
    # findpath(fire_lonlat, result)